# Larger systems

In this chapter we will consider some aspects which are important for considering larger systems. These include:

- Ways to lower computational costs (cost-saving measures)

- The inclusion of an environment

- ...

Most of the examples provided here are have been calculated on a larger cluster, so directly running the code on a laptop is not advised. They are here included as static core, with results taken from separate calculations.


## Valence-to-core XES

Here considering transition metal complex with low degree of static correlation, such that TDDFT and ADC can be used.

Find experimental measurements and consider using

1. Overlap

2. TDDFT (?)

3. ADC (?)


## X-ray absorption spectrum of solvated system

### Uracil with CVS-ADC(2)-x

Consider uracil (or some similar system) with CVS-ADC(2)-x. Consider subjects such as 

1. Tailor CVS space

2. Gas phase spectra

3. Spectra with PE

### Phthalocyanine with TDDFT

Consider phthalocyanine (or some similar system) with TDDFT. Focus on just making it work, so not necessarily including an environment.


## X-ray emission spectrum of solvated system

### Uracil with ADC(2)

Consider uracil (or some similar system) with ADC(2) / ADC(3/2).

1. Gas phase

2. Environment with PE

Also could discuss the impact of double-excitations, which may necessitate ADC(3/2)

### Phthalocyanine with DFT

Consider phthalocyanine (or some similar system) with TDDFT and DFT overlap (compare and contrast). Focus on just making it work, so not necessarily including an environment.


## Coupling to molecular dynamics

Maybe include this, although it may be tricky. Examples could include:

1. Dynamics + XAS

2. Dynamics + XES

3. Inclusion of environment

4. Core-hole dynamics with some model


## Transient X-ray spectroscopy

```{note}
A long-term goal, which might be given its own section.

- XAS: two-step approach and coupling
```